In [108]:
from scipy.stats import percentileofscore as pctrank
from pandas.io.formats.style import Styler
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import numpy as np
import sys, os

### Consts

In [2]:
DATE = "2010-01-01"

### Data

In [3]:
products = pd.read_csv("data/vol_products.csv")
products = products.set_index("Ticker")["Link"].to_dict()
products

{'RVOL': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL_History.csv',
 'RVOL3M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL3M_History.csv',
 'RVOL6M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL6M_History.csv',
 'RVOL12M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVOL12M_History.csv',
 'VIX': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VIXcurrent.csv',
 'VXD': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VXDohlcprices.csv',
 'VXN': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VXNcurrent.csv',
 'RVX': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/RVXdailyprices.csv',
 'VIX3M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VIX3Mdailyprices.csv',
 'VIX6M': 'http://www.cboe.com//publish/scheduledtask/mktdata/datahouse/VIX6Mdailyprices.csv'}

### Loaders

In [4]:
def rvol(key):
    rv = pd.read_csv(products[key])
    rv['Date'] = pd.to_datetime(rv.Date.values)
    rv = rv.sort_values('Date', ascending=True)
    rv.columns = ['date', 'val']
    return rv

def get_vixm(key):
    v = pd.read_csv(products[key], skiprows=2)
    v.columns = ['date', 'open', 'high', 'low', 'val']
    v['date'] = pd.to_datetime(v.date.values)
    return v[['date', 'val']]

def get_major(key, name, skip):
    v = pd.read_csv(products[key], skiprows=skip)
    v['Date'] = pd.to_datetime(v.Date.values)
    v = v[['Date', name]]
    v.columns = ['date', 'val']
    return v

### Downloads

In [5]:
rv, rv3m, rv6m = rvol("RVOL"), rvol("RVOL3M"), rvol("RVOL6M")

In [6]:
vix3m, vix6m = get_vixm("VIX3M"), get_vixm("VIX6M")

In [7]:
vix = get_major("VIX", "VIX Close", 1)
vxn = get_major("VXN", "Close", 2)
rvx = get_major("RVX", "Close", 2)
vxd = get_major("VXD", "Close", 4)

### Spot Indices

In [8]:
URL = "https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={p1}&period2={p2}&interval=1d&events=history&includeAdjustedClose=true"

dt = datetime.now()

p1 = datetime(dt.year-15, dt.month, dt.day)
p2 = datetime(dt.year, dt.month, dt.day)

p1 = int(p1.timestamp() / 1000) * 1000
p2 = int(p2.timestamp() / 1000) * 1000

In [9]:
def get_index(ticker, p1, p2):
    index = pd.read_csv(URL.format(ticker=ticker, p1=p1, p2=p2))[['Date', 'Adj Close']]
    index['Date'] = pd.to_datetime(index.Date.values)
    index.columns = ['date', 'val']
    return index[index.date >= DATE].reset_index(drop=True)

def calculate_rv(data, days):
    x = np.log(data.val / data.val.shift()) ** 2
    x = x.rolling(days, min_periods=1).sum()
    return np.sqrt(x * (252 / days)) * 100

In [10]:
spx = get_index("%5EGSPC", p1, p2)
rut = get_index("%5ERUT", p1, p2)
dji = get_index("%5EDJI", p1, p2)
ndx = get_index("%5ENDX", p1, p2)

In [11]:
spxrv = calculate_rv(spx, 21)
spxrv3m = calculate_rv(spx, 63)
spxrv6m = calculate_rv(spx, 126)

rutrv = calculate_rv(rut, 21)
ndxrv = calculate_rv(ndx, 21)
djirv = calculate_rv(dji, 21)

### Filter by Date

In [12]:
rv = rv[rv.date >= DATE].reset_index(drop=True)
rv3m = rv3m[rv3m.date >= DATE].reset_index(drop=True)
rv6m = rv6m[rv6m.date >= DATE].reset_index(drop=True)
vix = vix[vix.date >= DATE].reset_index(drop=True)
vix3m = vix3m[vix3m.date >= DATE].reset_index(drop=True)
vix6m = vix6m[vix6m.date >= DATE].reset_index(drop=True)
vxn = vxn[vxn.date >= DATE].reset_index(drop=True)
rvx = rvx[rvx.date >= DATE].reset_index(drop=True)
vxd = vxd[vxd.date >= DATE].reset_index(drop=True)

### Spreads

In [81]:
def spread_stats(ticker, ticker1, ticker2, rvol1, rvol2):
    
    data = pd.DataFrame(zip(
        ticker1.date,
        ticker1.val,
        ticker2.val,
        ticker1.val - ticker2.val,
        rvol1 - rvol2
    ), columns = ['date', 'v1', 'v2', 'spread', 'rvspread'])
    
    r3 = data.spread.rolling(252*3, min_periods=1)
    r1 = data.spread.rolling(252, min_periods=1)

    data['Ticker'] = ticker
    data['Carry'] = data.rvspread - data.spread
    data['Corr6M'] = data.v1.rolling(126, min_periods=1).corr(data.v2)
    data['Corr3M'] = data.v1.rolling(63, min_periods=1).corr(data.v2)
    
    data['Mean3Y'] = r3.mean()
    data['ZScore3Y'] = (data.spread - data.Mean3Y) / r3.std()
    data['Min3Y'] = r3.min()
    data['Max3Y'] = r3.max()
    data['Rank3Y'] = (data.spread - data.Min3Y) / (data.Max3Y - data.Min3Y)
    data['PctRank3Y'] = r3.apply(lambda x: pctrank(x, x.values[-1]) / 100)

    data['Mean'] = r1.mean()
    data['ZScore'] = (data.spread - data.Mean) / r1.std()
    data['Min'] = r1.min()
    data['Max'] = r1.max()
    data['Rank'] = (data.spread - data.Min) / (data.Max - data.Min)
    data['PctRank'] = r1.apply(lambda x: pctrank(x, x.values[-1]) / 100)
    
    return data

In [177]:
rvx_vix = spread_stats("RVX VIX", rvx, vix, rutrv, spxrv)
vxd_vix = spread_stats("VXD VIX", vxd, vix, djirv, spxrv)
vxn_vix = spread_stats("VXN VIX", vxn, vix, ndxrv, spxrv)
vix3m_vix = spread_stats("VIX3M VIX", vix3m, vix, spxrv3m, spxrv)
vix6m_vix = spread_stats("VIX6M VIX", vix6m, vix, spxrv6m, spxrv)
vxd_vxn = spread_stats("VXD VXN", vxd, vxn, djirv, ndxrv)
rvx_vxn = spread_stats("RVX VXN", rvx, vxn, rutrv, ndxrv)
rvx_vxd = spread_stats("RVX VXD", rvx, vxd, rutrv, djirv)

### Styling

In [178]:
def style(data):
    
    styler = Styler(data.iloc[-1:], precision=3)
    
    def z_score(x):
        m = x.mean()
        return m - x.std() * 3, m + x.std() * 3
    
    keys = ['Mean', '6M Corr.', '3M Corr.', 'Carry', 'Implied Spread', 'RVol Spread']
    for key in keys:
        l, h = z_score(data[key])
        styler = styler.background_gradient(cmap="bwr", vmin=l, vmax=h, subset=[key])
                
    keys = ['Rank', 'Pct. Rank']
    styler = styler.background_gradient(cmap="bwr", vmin=-0.1, vmax=1.1, subset=keys)
    styler = styler.background_gradient(cmap="bwr", vmin=-3, vmax=3, subset=["Z-Score"])
    
    return styler

def filter_cols(data, columns, new_cols):
    data = data[columns]
    data.columns = new_cols
    return data

In [179]:
lcols = [
    'Ticker', 'spread', 'rvspread', 'Carry', 'Corr6M', 'Corr3M',
    'Mean3Y', 'ZScore3Y', 'Min3Y', 'Max3Y', 'Rank3Y', 'PctRank3Y'
]

scols = [
    'Ticker', 'spread', 'rvspread', 'Carry', 'Corr6M', 'Corr3M',
    'Mean', 'ZScore', 'Min', 'Max', 'Rank', 'PctRank'
]
fcols = [
    'Ticker', "Implied Spread", "RVol Spread", "Carry", "6M Corr.", "3M Corr.",
    "Mean", "Z-Score", "Min", "Max", "Rank", "Pct. Rank"
]

### Merge

In [188]:
def merge(items):
    
    html = items[0]
    
    for item in items[1:]:
        html.find("style").insert_after(item.find("style"))
        html.find("tbody").append(item.find_all("tr")[1])
        
    return html

### Short Tables

In [194]:
items = [
    style(filter_cols(rvx_vix.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(vxd_vix.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(vxn_vix.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(vix3m_vix.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(vix6m_vix.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(vxd_vxn.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(rvx_vxn.copy(), scols, fcols)).hide_index().render(),
    style(filter_cols(rvx_vxd.copy(), scols, fcols)).hide_index().render()
]
items = [
    BeautifulSoup(item)
    for item in items
]
html_short = merge(items)

In [195]:
with open("/home/zquantz/Downloads/index.html", "w") as file:
    file.write(str(html_short))

### Long Tables

In [199]:
items = [
    style(filter_cols(rvx_vix.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(vxd_vix.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(vxn_vix.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(vix3m_vix.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(vix6m_vix.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(vxd_vxn.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(rvx_vxn.copy(), lcols, fcols)).hide_index().render(),
    style(filter_cols(rvx_vxd.copy(), lcols, fcols)).hide_index().render()
]
items = [
    BeautifulSoup(item)
    for item in items
]
html_long = merge(items)

In [200]:
with open("/home/zquantz/Downloads/index_long.html", "w") as file:
    file.write(str(html_long))

### Table Formatting

In [205]:
stable = html_short.find("table")
stable.attrs['class'] = "table table-sm table-hover"

In [206]:
ltable = html_long.find("table")
ltable.attrs['class'] = "table table-sm table-hover"

In [210]:
table_styles = ''.join(
    list(map(str, html_short.find_all("style"))) + 
    list(map(str, html_long.find_all("style")))
)

### HTML Email

In [231]:
page = """
    <!DOCTYPE html>
    <html>
    <head>

        <title>Volatility Monitor</title>

        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" crossorigin="anonymous">

        <!-- Font -->
        <link rel="preconnect" href="https://fonts.gstatic.com">
        <link href="https://fonts.googleapis.com/css2?family=Ubuntu&display=swap" rel="stylesheet">

        <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js" integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj" crossorigin="anonymous"></script>
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ho+j7jyWK8fNQe+A12Hb8AhRq26LrZ/JpcUGGOn+Y7RsweNrtN/tE3MoK7ZeZDyx" crossorigin="anonymous"></script>

    </head>
    <body>

        <style type="text/css">

            .jumbotron {
                background-image: url(https://media.giphy.com/media/S8IIUwihhFu7O90I8l/giphy.gif);
                background-size: 20rem;
                background-repeat: no-repeat;
                background-color: black;
                background-position: right;
            }
            .jumbotronHeader {
                font-family: 'Ubuntu', sans-serif;
                color: white;
                font-size: 5rem
            }
            .jumbotronLabel {
                color:white;
                font-size: 2rem;
                font-family: 'Ubuntu', sans-serif;
                padding-top: 1rem
            }
            .list-group-item.active {
                background-color: black;
                border-color: black;
                color:white;
            }
            .statButton {
                text-align: center;
            }

        </style>
        TABLE_STYLES_HERE

        <div class="container-fluid" style="max-width: 50%; padding:0;">

            <div class="jumbotron jumbotron-fluid" style="padding-left: 3rem; padding-right: 3rem">

                <div class="container">

                    <h1 class="jumbotronHeader">
                        <b>O p t i Q s</b>
                    </h1>
                    <h6 class="jumbotronLabel">
                        - Vol Monitor
                    </h6>

                </div>

            </div>

            <div class="row" style="margin-right: 0; margin-left: 0; margin-bottom: 1rem; display: block; width: 30%;">

                    <div class="list-group list-group-horizontal" id="list-tab" role="tablist">
                        <a class="list-group-item list-group-item-action statButton active" id="list-1ystats" data-toggle="list" href="#list-1y" role="tab" aria-controls="home">1 Yr Stats</a>
                        <a class="list-group-item list-group-item-action statButton" id="list-3ystats" data-toggle="list" href="#list-3y" role="tab" aria-controls="profile">3 Yr Stats</a>
                    </div>
                    
            </div>
            
            <div class="row" style="margin-right: 0rem; margin-left: 0;">
            
                <div class="col-12-xl col-12-lg col-12-md col-12-sm">
                    <div class="tab-content" id="nav-tabContent">
                        <div class="tab-pane fade show active" id="list-1y" role="tabpanel" aria-labelledby="list-1ystats">
                            SHORT_TABLE_HERE
                        </div>
                        <div class="tab-pane fade" id="list-3y" role="tabpanel" aria-labelledby="list-3ystats">
                            LONG_TABLE_HERE
                        </div>
                    </div>
                </div>
                
            </div>

        </div>

    </body>
    </html>
"""
page = page.replace("LONG_TABLE_HERE", str(ltable))
page = page.replace("SHORT_TABLE_HERE", str(stable))
page = page.replace("TABLE_STYLES_HERE", str(table_styles))
with open("/home/zquantz/Documents/Quant/OptionsResearch/VolDashboard/auto.html", "w") as file:
    file.write(page)